# Creating a Discussion

The simplest task you can accomplish with this library is to create a small discussion between LLMs. 

This guide will teach you the basic setup of the library. You will understand how to setup models, user-agents and how to coordinate them in a discussion. By the end of htis guide, you will be able to run a small discussion with a moderator and save it to the disk for persistence.

## Basics

### The Model

SynDisco can theoretically support any LLM, as long as it is wrapped in a `BaseModel` wrapper. The `BaseModel` class is a very simple interface with one method. This method gives the underlying LLM input, and returns its output to the library.

There already exists a `TransformersModel` class which handles models from the `transformers` python library. In 90% of your applications, this will be enough. We can load a TransformersModel using the following code:

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from syndisco.model import TransformersModel

llm = TransformersModel(
    model_path="unsloth/Llama-3.2-3B-Instruct-bnb-4bit",
    name="test_model",
    max_out_tokens=100,
)

/home/dimits/miniconda3/envs/syndiscooooo/lib/python3.13/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(
Device set to use cuda:0


This will download a small LLM from huggingface. You can substitute the model_path for any similar model in [HuggingFace](https://huggingface.co/) supporting the Transformers library.

### Creating personas

All `actors` can be defined by a `persona`, aka a set of attributes that define them. These attributes can be age, ethnicity, and even include special instructions on how they should behave.

Creating a persona programmatically is simple:

In [3]:
from syndisco.actors import Persona

persona_data = [
    {
        "username": "Emma35",
        "age": 38,
        "sex": "female",
        "education_level": "Bachelor's",
        "sexual_orientation": "Heterosexual",
        "demographic_group": "Latino",
        "current_employment": "Registered Nurse",
        "special_instructions": "",
        "personality_characteristics": [
            "compassionate",
            "patient",
            "diligent",
            "overwhelmed",
        ],
    },
    {
        "username": "Giannis",
        "age": 21,
        "sex": "male",
        "education_level": "College",
        "sexual_orientation": "Pansexual",
        "demographic_group": "White",
        "current_employment": "Game Developer",
        "special_instructions": "",
        "personality_characteristics": [
            "strategic",
            "meticulous",
            "nerdy",
            "hyper-focused",
        ],
    },
]

personas = [Persona(**data) for data in persona_data]

for persona in personas:
    print(persona)

{"username": "Emma35", "age": 38, "sex": "female", "sexual_orientation": "Heterosexual", "demographic_group": "Latino", "current_employment": "Registered Nurse", "education_level": "Bachelor's", "special_instructions": "", "personality_characteristics": ["compassionate", "patient", "diligent", "overwhelmed"]}
{"username": "Giannis", "age": 21, "sex": "male", "sexual_orientation": "Pansexual", "demographic_group": "White", "current_employment": "Game Developer", "education_level": "College", "special_instructions": "", "personality_characteristics": ["strategic", "meticulous", "nerdy", "hyper-focused"]}


Since creating a lot of distinct users is essential in running large-scale experiments, users are usually defined in JSON format. That way, you can change anything without touching your code!

[Here](https://github.com/dimits-ts/synthetic_moderation_experiments/blob/master/data/discussions_input/personas/personas.json) is an applied example of how to mass-define user personas through JSON files.

### Creating the user-agents

Having a `persona` and a `model` we can finally create an `actor`. The actor will personify the selected persona using the model to talk.

Besides a persona and a model, the actors will also need instructions and a context. By convention, all actors share the same context, and all user-agents share the same instructions. Personalized instructions are defined in the actor's persona.

In [4]:
from syndisco.actors import Actor, ActorType


CONTEXT = "You are taking part in an online conversation"
INSTRUCTIONS = "Act like a human would"

actors = [
    Actor(
        model=llm,
        persona=p,
        actor_type=ActorType.USER,
        context=CONTEXT,
        instructions=INSTRUCTIONS
    )
    for p in personas
]

### Managing turn-taking

In real-life discussions, who gets to speak at each point in time is determined by complex social dynamics, which are difficult to realistically model.
However, there are ways with which we can simulate these dynamics. 

SynDisco uses the `TurnManager` class to model turn taking. Two implementations are available by default: Round Robin, and Random Weighted

- `Round Robin` is the simplest, most intuitive way to model turn-taking; everyone gets to talk once per round. Once everyone talks once, they get to talk again in the same sequence

In [5]:
from syndisco.turn_manager import RoundRobin


rrobin_turn_manager = RoundRobin(["John", "Mary", "Howard", "Todd"])

for i in range(10):
    print(next(rrobin_turn_manager))

John
Mary
Howard
Todd
John
Mary
Howard
Todd
John
Mary


- `RandomWeighted` on the other hand throws a weighted coin on each round. If the coin flip succedes, the previous user gets to respond. If not, another user is selected at random

In [6]:
from syndisco.turn_manager import RandomWeighted


rweighted_turn_manager = RandomWeighted(
    names=["John", "Mary", "Howard", "Todd"], p_respond=0.5
)

for i in range(10):
    print(next(rweighted_turn_manager))

Todd
Mary
Howard
John
Todd
Mary
Todd
Howard
Mary
Howard


## Generating a discussion

Let's start with the most basic task; a single discussion between two user-agents.

Since we only have two users, a RoundRobin approach where each user takes a turn sequentially is sufficient.

Now we can run a simple discussion

In [7]:
from syndisco.jobs import Discussion


turn_manager = RoundRobin([actor.get_name() for actor in actors])
conv = Discussion(next_turn_manager=turn_manager, users=actors)
conv.begin()

User Emma35 posted:
I'm so glad I finally got a moment to myself. I've been running around
nonstop since the morning shift at the hospital. How about you, how's
your day going? 

User Giannis posted:
"Hey Emma, I feel you. I've been in coding mode nonstop since morning
too. Just trying to meet a tight deadline for a project I'm working
on. But I did manage to squeeze in a quick lunch break and catch up on
some gaming. How was your shift? Anything exciting happen?" 

User Emma35 posted:
"Ah, I'm glad I could finally take a breather too! My shift was pretty
crazy, as usual. We had a code blue in the ER and I had to help
stabilize the patient, but thankfully everything turned out okay. It
was a bit overwhelming, to be honest, but it was a great feeling
knowing I was able to help. I'm just ready to unwind now and maybe
catch up on some sleep. How was your coding marathon? Did you manage
to meet that deadline?" 

User Giannis posted:
"Ah, nice to hear that everything turned out okay at the 

Let's add a moderator to oversee the discussion.

In [8]:
MODERATOR_INSTRUCTIONS = "You are a moderator. Oversee the discussion"

moderator_persona = Persona(
    **{
        "username": "Moderator",
        "age": 41,
        "sex": "male",
        "education_level": "PhD",
        "sexual_orientation": "Pansexual",
        "demographic_group": "White",
        "current_employment": "Moderator",
        "special_instructions": "",
        "personality_characteristics": [
            "strict",
            "neutral",
            "just",
        ],
    }
)

moderator = Actor(
    model=llm,
    persona=moderator_persona,
    actor_type=ActorType.USER,
    context=CONTEXT,
    instructions=MODERATOR_INSTRUCTIONS
)

# remember to update this!
turn_manager = RoundRobin(
    [actor.get_name() for actor in actors] + [moderator.get_name()]
)
conv = Discussion(
    next_turn_manager=turn_manager,
    users=actors + [moderator],
    moderator=moderator,
)
conv.begin()

User Emma35 posted:
I'm happy to chat with you, but I don't see what Emma35 posted. Could
you please share the text or topic you'd like to discuss? I'm all
ears! 

User Moderator posted:
User Moderator posted: Hello Emma35, I'm glad you're excited to chat.
Unfortunately, this is the beginning of our conversation, and there is
no previous post from you to refer to. Would you like to start fresh
and introduce a topic you'd like to discuss, or would you like me to
suggest some conversation starters? 

User Giannis posted:
Thanks for the warm welcome, Moderator! I'm excited to start fresh. I
think I'd love to discuss something that's been on my mind lately -
have you guys ever heard of the latest advancements in artificial
intelligence in game development? I've been working on a project that
incorporates some pretty cutting-edge AI techniques, and I'm curious
to hear what others think about the possibilities and challenges that
come with it. What are your thoughts? 

User Moderator posted:

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


User Moderator posted:
User Giannis posted: I'm glad you're interested, Emma! I'm actually
working on a new game that incorporates machine learning algorithms to
create a more immersive experience for players. One of the key
features I'm experimenting with is procedural generation, which uses
AI to create unique levels and game worlds on the fly. I'm also
looking into using natural language processing to enable more
realistic NPC interactions. I'd love to hear your thoughts on the
potential benefits and challenges of using 

User Moderator posted:
User Giannis posted:   I'm glad to hear that you're interested, Emma!
Procedural generation is a game-changer for game development. The
ability to generate unique levels and game worlds on the fly can
create an almost endless variety of experiences for players. However,
it also presents some challenges, such as ensuring that the generated
content is coherent and fun to play through. Have you come across any
notable examples of procedural gene

### Saving a discussion to the disk

It's best practice to save the results of each discussion after it has concluded. This way, no matter what happens to the program executing the discussions, progress will be checkpointed.

The `Discussion` class provides a method for saving its logs and related metadata to a JSON file:

In [9]:
import tempfile
import json

tp = tempfile.NamedTemporaryFile(delete=True)

conv.to_json_file(tp.name)

# if you are running this on Windows, uncomment this line
# tp.close()
with open(tp.name, mode="rb") as f:
    print(json.dumps(json.load(f), indent=2))

{
  "id": "98f03c72-b8e6-49aa-9c92-f6383d575ab4",
  "timestamp": "25-06-12-15-20",
  "users": [
    "Emma35",
    "Giannis",
    "Moderator"
  ],
  "moderator": "Moderator",
  "user_prompts": [
    {
      "context": "You are taking part in an online conversation",
      "instructions": "Act like a human would",
      "type": "1",
      "persona": {
        "username": "Emma35",
        "age": 38,
        "sex": "female",
        "sexual_orientation": "Heterosexual",
        "demographic_group": "Latino",
        "current_employment": "Registered Nurse",
        "education_level": "Bachelor's",
        "special_instructions": "",
        "personality_characteristics": [
          "compassionate",
          "patient",
          "diligent",
          "overwhelmed"
        ]
      }
    },
    {
      "context": "You are taking part in an online conversation",
      "instructions": "Act like a human would",
      "type": "1",
      "persona": {
        "username": "Giannis",
        "age"

Congratulations, you can now run fully synthetic discussions! You may want to experiment with adding more than 2 users or testing more realistic turn taking procedures (for example, check out the `RandomWeighted` turn manager).